### Crack Classification with Deep Learning

Deep Learning (DL) is a subset of Machine Learning that uses Neural Network inspired architecture to make predictions. Convolutional Neural Networks (CNN) are a type of DL model that is effective in learning patterns in 2-dimensional data such as images. Images of concrete with and without cracks are used to train a classifier to identify cracks. See [Machine Learning for Engineers course website](https://apmonitor.com/pds) for the source code to [Image Classification: Bits and Cracks](https://apmonitor.com/pds/index.php/Main/BitClassification). The webpage starts with a tutorial on bit classification and then moves to crack classification.

<img align=left width=400px src='https://apmonitor.com/pds/uploads/Main/bit_classification.png'>

### Setup

Import the following Python modules. Use ```pip``` to install any missing packages. For example, if there is an error: ```ModuleNotFoundError: No module named 'cv2'```, add a new cell and run the following command in another cell: ```pip install opencv-python```. An online search *how to install python cv2* may be needed if the name of the install package isn't the same as the import name. See additional information on [how to install and manage Python packages](https://apmonitor.com/pds/index.php/Main/InstallPythonPackages).

In [ ]:
import os
import zipfile
import urllib.request
import cv2
import re
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
#pip install opencv-python

### Labeled Photos

Download photos from [Crack Images (concrete_cracks.zip)](http://apmonitor.com/pds/uploads/Main/concrete_crack.zip). Download and extract the images by either selecting the [link](http://apmonitor.com/pds/uploads/Main/concrete_cracks.zip) or running the code cell below to download and unzip the archive.

<img align=left width=400px src='https://apmonitor.com/pds/uploads/Main/concrete_cracks.png'>

In [ ]:
# download concrete_cracks.zip
file = 'concrete_cracks.zip'
url = 'http://apmonitor.com/pds/uploads/Main/'+file
urllib.request.urlretrieve(url, file)

# extract archive and remove concrete_cracks.zip
with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall('./')
os.remove(file)

The archive contains two folders, a test folder and train folder with subdirectories corresponding to the possible image types (Positive or Negative). The images are found within each subdirectory. The tree structure of the folders is:

```
  ├───test
  │   ├───Positive
  │   └───Negative
  └───train
      ├───Positive
      └───Negative
```

Import the photos into the Python session. The first step is to process the images into a format that 1) makes the data readable to the model, and 2) provides more training material for the model to learn. For example, the ```train_processor``` variable scales the data so that it can be a feature (input) for the model, but also takes each images and augments it so that the model can learn from multiple variations of the same picture. It flips it horizontally, rotates it, and shifts it, and more to make sure the model learns from the shape of the crack rather than the orientation or size. 

**Change:** The original image is 227x277 and the image is rescaled to 128x128 without significant loss in classifier accuracy. The [solution video](https://youtu.be/UotUJuBb0qc) uses 227x227 but there is an error with later versions of TensorFlow that the image pixels must be a multiple of the layer size.  

In [ ]:
# Data processing
train_processor = ImageDataGenerator(rescale = 1./255, \
    horizontal_flip = True, zoom_range = 0.2,          \
    rotation_range = 10, shear_range = 0.2,            \
    height_shift_range = 0.1, width_shift_range = 0.1)
test_processor = ImageDataGenerator(rescale = 1./255)

# Load data
train = train_processor.flow_from_directory('train',   \
    target_size = (128,128), batch_size = 32,         \
    class_mode = 'categorical', shuffle = True)

test = test_processor.flow_from_directory('test',      \
    target_size = (128,128), batch_size = 32,         \
    class_mode = 'categorical', shuffle = False)

### Model Building

Next step is to build the CNN model with options for building the model. This includes the number of convolutional layers, fully connected dense layers, the number of nodes in each layer, and the number of training epochs. For more information on these parameters and Convolutional Neural Networks in general, see [Computer Vision with Deep Learning](https://apmonitor.com/pds/index.php/Main/VisionDeepLearning).  Change these parameters and analyze the performance of the model.

In [ ]:
# choose model parameters
num_conv_layers = 2
num_dense_layers = 1
layer_size = 16
num_training_epochs = 15

In [ ]:
# Initiate model variable
model = Sequential()

# begin adding properties to model variable
# e.g. add a convolutional layer
model.add(Conv2D(layer_size, (3,3), input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# add additional convolutional layers based on num_conv_layers
for _ in range(num_conv_layers-1):
    model.add(Conv2D(layer_size, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

# reduce dimensionality
model.add(Flatten())

# add fully connected "dense" layers if specified
for _ in range(num_dense_layers):
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# add output layer
model.add(Dense(2))
model.add(Activation('softmax'))

# compile the sequential model with all added properties
model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'],
                )

# use the data already loaded previously to train/tune the model
model.fit(train,
            epochs=num_training_epochs,
            validation_data = test)

# save the trained model
model.save(f'cracks.h5')

### Model Testing

The model is trained and saved as an `h5` file. The last line of the printed output contains the accuracy for both the training and testing data.

```
Epoch 14/15
2/2 [====] - 3s 2s/step - loss: 0.8785 - accuracy: 0.6202 - val_loss: 0.9091 - val_accuracy: 0.6000
Epoch 15/15
2/2 [====] - 3s 2s/step - loss: 0.9028 - accuracy: 0.5721 - val_loss: 0.8648 - val_accuracy: 0.6667
```

The ```val_accuracy``` is the accuracy on the test images that are not included in the training. Hyperparameter optimization can be used to improve the accuracy by adjusting the CNN architecture, training selections, or other parameters. The function ```make_prediction``` takes the file path to a crack photo as an input and produces a classification result.

In [ ]:
btype = ['Negative','Positive'] # possible output values
def make_prediction(image_fp):
    im = cv2.imread(image_fp) # load image
    plt.imshow(im)
    img = image.load_img(image_fp, target_size = (128,128))
    img = image.img_to_array(img)

    image_array = img / 255. # scale the image
    img_batch = np.expand_dims(image_array, axis = 0)
    
    predicted_value = btype[model.predict(img_batch).argmax()]
    true_value = re.search(r'(Negative)|(Positive)', image_fp)[0]
    
    out = f"""Predicted Crack Type: {predicted_value}
    True Crack Type: {true_value}
    Correct?: {predicted_value == true_value}"""
    
    return out

In [ ]:
# randomly select type (0-1) and image number (1-5)
i = random.randint(0,1); j = random.randint(19901,20000)
b = btype[i]; im = str(j) + '.jpg'
test_image_filepath = r'./test/'+b+'/'+im
print(make_prediction(test_image_filepath))

The validation accuracy as well as individual testing shows that there are misclassifications.

Here are a few things that can improve the accuracy for this application:  
- <b>More photos!</b> This is the most important thing, Machine Learning typically requires many photos that are representative of what the classifiers see. At this point, there are not nearly enough photos for the model to learn each crack type.
- <b>Background Clutter.</b> Most of the images in this set have the background removed. To train a classifier to identify crack types in the field, more photos are needed.
- <b>Hyperparameter Optimization.</b> To make the best model, the best parameters must be selected to maximize the accuracy (hyperparameter optimization). Packages such as [Hyperopt](http://hyperopt.github.io/hyperopt/) can intelligently try different parameter combinations to increase the accuracy without more data.